<a href="https://colab.research.google.com/github/zhangxs131/paper_demo_for_fun/blob/main/simCTG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SimCTG

腾讯AI lab 通过对GPT2训练过程中，加入了对比损失函数和对比搜索进行解码，缓解了GPT2的各向异性，减少了重复语句出现。

##安装依赖

In [1]:
!git clone https://github.com/yxuansu/simctg.git
%cd simctg
!pip install -r requirements.txt

Cloning into 'simctg'...
remote: Enumerating objects: 1413, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1413 (delta 37), reused 0 (delta 0), pack-reused 1344
Receiving objects: 100% (1413/1413), 6.43 MiB | 18.45 MiB/s, done.
Resolving deltas: 100% (846/846), done.
/content/simctg
     |████████████████████████████████| 60 kB 3.5 MB/s 
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 5.9 MB 54.1 MB/s 
     |████████████████████████████████| 2.5 MB 54.4 MB/s 
     |████████████████████████████████| 1.2 MB 54.5 MB/s 
     |████████████████████████████████| 880 kB 53.1 MB/s 
     |████████████████████████████████| 3.3 MB 49.2 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=e08b5acf8037c1ebdb818ac013e3e4ef46d30c0a0382018229c529426dc5cf8a
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0

##汉语示例

In [2]:
import torch
import sys
sys.path.append(r'./pretraining')
from simctg import SimCTGPretraining

model_path = r'uer/gpt2-chinese-cluecorpussmall'
model = SimCTGPretraining(model_path)
model.eval()

#准备prefix input
text='科比布莱恩特'
tokens=model.tokenizer.tokenize(text)
input_ids=model.tokenizer.convert_tokens_to_ids(tokens)
input_ids=torch.LongTensor(input_ids).view(1,-1)

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

Further pre-train with available parameters.


Downloading:   0%|          | 0.00/421M [00:00<?, ?B/s]

###使用Contrastive search解码生成

In [3]:
beam_width, alpha, decoding_len = 3, 0.6, 128
eos_token = '[SEP]'
print (model.fast_contrastive_search(input_ids, beam_width, alpha, decoding_len, eos_token))

科比布莱恩特在接受采访时表示自己很喜欢篮球，[UNK]我觉得我的生活很美好，这是一种享受。


###使用核采样进行解码生成结果

In [4]:
nucleus_p, decoding_len = 0.95, 128
eos_token = '[SEP]'
print (model.nucleus_sampling(input_ids, nucleus_p, decoding_len, eos_token))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


科比布莱恩特看了他的报道后直夸比基·布莱恩特绝不会让别人失望，曼联队是全联盟中有球员有实力的球队，但他不会给国内球员充分表现，那只是媒体上炒出来的一个笑话。


###使用greedy search 解码

In [5]:
decoding_len = 128
eos_token = '[SEP]'
print (model.greedy_search(input_ids, decoding_len, eos_token))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


科比布莱恩特，他们的表现都很出色，但是他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中表现不佳，他们的球队在季后赛中


###仅使用beam search进行解码

In [8]:
beam_width, decoding_len = 12, 128
eos_token = '[SEP]'
print (model.beam_search(input_ids, 10, decoding_len, eos_token))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


科比布莱恩特说道：[UNK]我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道，我不知道


感觉这greedy  search和beam search效果也太差了，我正常用GPT2时候也没这么低，感觉这弄得不太行

##英语示例

In [9]:
import torch
import sys
sys.path.append(r'./pretraining')
from simctg import SimCTGPretraining
# load SimCTG model pretrained on the large-scale Wikipedia corpus
model_path = r'cambridgeltl/simctg_english_wikipedia'
model = SimCTGPretraining(model_path)
model.eval()


text = r'When you break up with your girlfriend,'
tokens = model.tokenizer.tokenize(text)
input_ids = model.tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.LongTensor(input_ids).view(1,-1)

# use contrastive search to generate the result
beam_width, alpha, decoding_len = 5, 0.6, 128
eos_token = '<|endoftext|>'
print (model.fast_contrastive_search(input_ids, beam_width, alpha, decoding_len, eos_token))

Downloading:   0%|          | 0.00/877 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/218 [00:00<?, ?B/s]

Further pre-train with available parameters.


Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

When you break up with your girlfriend, it's hard to be in a relationship".


##日语示例

In [ ]:
import torch
import sys
sys.path.append(r'./pretraining')
from simctg import SimCTGPretraining
# load an off-the-shelf Japanese GPT (https://huggingface.co/colorfulscoop/gpt2-small-ja)
model_path = r'colorfulscoop/gpt2-small-ja'
model = SimCTGPretraining(model_path)
model.eval()

In [13]:
#输入日语句子，这里选了句海贼王，台词，这世界全部掌握的男人
text = r'この世の全てを手に入れた男'
tokens = model.tokenizer.tokenize(text)
input_ids = model.tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.LongTensor(input_ids).view(1,-1)

In [14]:
# (1) use contrastive search to generate the result
beam_width, alpha, decoding_len = 5, 0.6, 128
eos_token = model.tokenizer.eos_token
print (model.fast_contrastive_search(input_ids, beam_width, alpha, decoding_len, eos_token))


この世の全てを手に入れた男の名を刻んだ石碑が建つ。この世を滅ぼすために悪霊を追い払おうとする者たちを退治し、世界の終わりを予感させるのである。アステカ暦では月の運行が止まる日が太陽暦と異なっていたため、その日の出と日没を基準に暦が調整された。この世の全ての物々交換の方法が神聖なものとされ、その儀式は「カトリポカ」と呼ばれた。アステカ暦の年中行事である祭日は、その日が日没のちょうど1週間前である12月31日(ユリウス暦では12月31日)に


翻译过来：
有一座石碑，上面刻着获得世间一切的人的名字。它消灭那些企图驱除邪灵以毁灭世界的人，让我们感到世界末日。在阿兹特克历中，月亮停止运行的日子与太阳历不同，所以日历是根据日出和日落来调整的。世间一切以物易物的方法都是神圣的，这种仪式被称为“Katripoka”。阿兹特克的全年假期是在 12 月 31 日（儒略历中的 12 月 31 日），正好是日落前一周。

In [15]:
# (2) use nucleus sampling to generate the result
nucleus_p, decoding_len = 0.95, 128
eos_token = model.tokenizer.eos_token
print (model.nucleus_sampling(input_ids, nucleus_p, decoding_len, eos_token))

この世の全てを手に入れた男がいた。見かけは心優しい王子がそのことを聴き、彼を捕えた。そして彼はすぐに宮殿を出て家に帰ることになる。そして暗黒の母の前から姿を消して姿を消し、残された人々はヴォルテールと会話して天国への道を共にすることになる。アトムワールドでは、地獄から生まれるアトムと混沌の塔が最大の被害を受けており、自らがともに死ぬことを選んだアトムが巨人の王となった。この作品ではアトムの生みの母(彼らは修道院に入り、七つの塔が完全に破壊された後に塔に住むようになった)と、他の「荒野の巨人」に対し塔を攻めるなどの善戦をしている。ヴォルテールの神官グリアースは


In [16]:
# (3) use greedy search to generate the result
decoding_len = 128
eos_token = model.tokenizer.eos_token
print (model.greedy_search(input_ids, decoding_len, eos_token))

この世の全てを手に入れた男であり、すべてのものを完全に失ったのだと語った。そして、それを証明するために、自分が「アプロディテの男」であると言うことにした。そして、私は自分の妻であるエマ(Emma)から「アプロディテの男」の肖像画を見せられる。その後に彼は「アルキメデスの帰還」(SalixAprides)の絵の具を分け与えるために、彼が持っていた宝石をすべて持ち去ってしまう。その後すぐに、アプロディテの男は「アルキメデスの帰還」の絵に描かれるアルキメデスの姿で現れる。そして、彼はアルキメデス


In [17]:
# (4) use beam search to generate the result
beam_width, decoding_len = 10, 128
eos_token = model.tokenizer.eos_token
print (model.beam_search(input_ids, 10, decoding_len, eos_token))

この世の全てを手に入れた男」と表現した。また、この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男」と表現した。この世の全てを手に入れた男を「永遠の男
